In [1]:
import pandas as pd
from win32evtlogutil import langid

books = pd.read_csv("books_with_categories.csv")

In [2]:
import tensorflow as tf
print("Built with CUDA:", tf.test.is_built_with_cuda())
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))
print("GPU Devices:", tf.config.list_physical_devices('GPU'))


Built with CUDA: False
Num GPUs Available: 0
GPU Devices: []


In [3]:
# emotion classification model

from transformers import pipeline
classifier = pipeline("text-classification",
                      model="j-hartmann/emotion-english-distilroberta-base",
                      top_k=None,
                      device=0)
classifier("I love this!")

Device set to use cpu


[[{'label': 'joy', 'score': 0.9771687984466553},
  {'label': 'surprise', 'score': 0.008528691716492176},
  {'label': 'neutral', 'score': 0.0057646059431135654},
  {'label': 'anger', 'score': 0.004419783595949411},
  {'label': 'sadness', 'score': 0.002092393347993493},
  {'label': 'disgust', 'score': 0.001611992483958602},
  {'label': 'fear', 'score': 0.00041385178337804973}]]

In [4]:
books["description"][0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [5]:
classifier(books["description"][0])

[[{'label': 'fear', 'score': 0.654841423034668},
  {'label': 'neutral', 'score': 0.1698516309261322},
  {'label': 'sadness', 'score': 0.1164093017578125},
  {'label': 'surprise', 'score': 0.020700573921203613},
  {'label': 'disgust', 'score': 0.019100699573755264},
  {'label': 'joy', 'score': 0.015161281451582909},
  {'label': 'anger', 'score': 0.003935145679861307}]]

In [6]:
classifier(books["description"][0].split())

[[{'label': 'neutral', 'score': 0.3978913724422455},
  {'label': 'fear', 'score': 0.13037142157554626},
  {'label': 'sadness', 'score': 0.12430447340011597},
  {'label': 'disgust', 'score': 0.1119837537407875},
  {'label': 'anger', 'score': 0.09650123864412308},
  {'label': 'surprise', 'score': 0.09280820935964584},
  {'label': 'joy', 'score': 0.04613954946398735}],
 [{'label': 'neutral', 'score': 0.9145249128341675},
  {'label': 'surprise', 'score': 0.03115367703139782},
  {'label': 'sadness', 'score': 0.02057531848549843},
  {'label': 'disgust', 'score': 0.013939082622528076},
  {'label': 'anger', 'score': 0.009547268971800804},
  {'label': 'joy', 'score': 0.007127388846129179},
  {'label': 'fear', 'score': 0.003132424084469676}],
 [{'label': 'neutral', 'score': 0.4886752963066101},
  {'label': 'surprise', 'score': 0.41193774342536926},
  {'label': 'disgust', 'score': 0.04322948306798935},
  {'label': 'anger', 'score': 0.0333968847990036},
  {'label': 'joy', 'score': 0.01030802167952

In [7]:
sentences = books["description"][0].split(".")
prediction = classifier(sentences)

In [9]:
sentences[0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives'

In [10]:
prediction[0]

[{'label': 'surprise', 'score': 0.7296028733253479},
 {'label': 'neutral', 'score': 0.14038564264774323},
 {'label': 'fear', 'score': 0.06816212832927704},
 {'label': 'joy', 'score': 0.04794241115450859},
 {'label': 'anger', 'score': 0.009156353771686554},
 {'label': 'disgust', 'score': 0.0026284768246114254},
 {'label': 'sadness', 'score': 0.002122159581631422}]

In [11]:
sentences[3]

' Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist'

In [12]:
prediction[3]

[{'label': 'fear', 'score': 0.928168535232544},
 {'label': 'anger', 'score': 0.032190706580877304},
 {'label': 'neutral', 'score': 0.012808662839233875},
 {'label': 'sadness', 'score': 0.008756856434047222},
 {'label': 'surprise', 'score': 0.008597890846431255},
 {'label': 'disgust', 'score': 0.008431797847151756},
 {'label': 'joy', 'score': 0.0010455806041136384}]

In [13]:
sorted(prediction[0], key=lambda x: x["label"])

[{'label': 'anger', 'score': 0.009156353771686554},
 {'label': 'disgust', 'score': 0.0026284768246114254},
 {'label': 'fear', 'score': 0.06816212832927704},
 {'label': 'joy', 'score': 0.04794241115450859},
 {'label': 'neutral', 'score': 0.14038564264774323},
 {'label': 'sadness', 'score': 0.002122159581631422},
 {'label': 'surprise', 'score': 0.7296028733253479}]

In [14]:
import numpy as np


emotion_lables = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
isbn = []
emotion_scores = {label: [] for label in emotion_lables}

def calculate_max_emotion_scores(predictions):
    per_emotion_scores = {label: [] for label in emotion_lables}
    for prediction in predictions:
        sorted_predictions = sorted(prediction, key=lambda x: x["label"])
        for index, label in enumerate(emotion_lables):
            per_emotion_scores[label].append(sorted_predictions[index]["score"])
    return {label: np.max(scores) for label, scores in per_emotion_scores.items()}


In [17]:
for i in range(10):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_lables:
        emotion_scores[label].append(max_scores[label])

In [18]:
emotion_scores

{'anger': [0.06413351744413376,
  0.6126193404197693,
  0.06413351744413376,
  0.35148346424102783,
  0.08141236007213593,
  0.2322256714105606,
  0.5381843447685242,
  0.06413351744413376,
  0.30066972970962524,
  0.06413351744413376],
 'disgust': [0.2735910415649414,
  0.34828561544418335,
  0.10400653630495071,
  0.1507221907377243,
  0.18449485301971436,
  0.727173924446106,
  0.1558549851179123,
  0.10400653630495071,
  0.2794819474220276,
  0.17792688310146332],
 'fear': [0.928168535232544,
  0.9425276517868042,
  0.9723208546638489,
  0.36070701479911804,
  0.09504329413175583,
  0.0513627827167511,
  0.7474280595779419,
  0.40449708700180054,
  0.9155243635177612,
  0.0513627827167511],
 'joy': [0.9327965974807739,
  0.7044214606285095,
  0.7672373652458191,
  0.2518816292285919,
  0.04056438431143761,
  0.04337580129504204,
  0.8725656270980835,
  0.04056438431143761,
  0.04056438431143761,
  0.04056438431143761],
 'sadness': [0.6462167501449585,
  0.8879393339157104,
  0.5494

In [20]:
from tqdm import tqdm

emotion_lables = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
isbn = []
emotion_scores = {label: [] for label in emotion_lables}


for i in tqdm(range(len(books))):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_lables:
        emotion_scores[label].append(max_scores[label])

100%|██████████| 5197/5197 [04:33<00:00, 19.01it/s]


In [21]:
emotions_df = pd.DataFrame(emotion_scores)
emotions_df["isbn13"] = isbn

In [22]:
emotions_df

,anger,disgust,fear,joy,sadness,surprise,neutral,isbn13
0,0.064134,0.273591,0.928169,0.932797,0.646217,0.967158,0.729603,9780002005883
1,0.612619,0.348286,0.942528,0.704421,0.887939,0.111690,0.252546,9780002261982
2,0.064134,0.104007,0.972321,0.767237,0.549477,0.111690,0.078765,9780006178736
3,0.351483,0.150722,0.360707,0.251882,0.732686,0.111690,0.078765,9780006280897
4,0.081412,0.184495,0.095043,0.040564,0.884389,0.475881,0.078765,9780006280934
...,...,...,...,...,...,...,...,...
5192,0.148208,0.030643,0.919165,0.255172,0.853722,0.980877,0.030656,9788172235222
5193,0.064134,0.114382,0.051363,0.400262,0.883199,0.111690,0.227765,9788173031014
5194,0.009997,0.009929,0.339219,0.947779,0.375754,0.066685,0.057625,9788179921623
5195,0.064134,0.104007,0.459269,0.759456,0.951104,0.368110,0.078765,9788185300535


In [23]:
books = pd.merge(books, emotions_df, on="isbn13")

In [24]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,...,title_and_subtitle,tagged_description,simple_categories,anger,disgust,fear,joy,sadness,surprise,neutral
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,...,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction,0.064134,0.273591,0.928169,0.932797,0.646217,0.967158,0.729603
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,...,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction,0.612619,0.348286,0.942528,0.704421,0.887939,0.111690,0.252546
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,...,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction,0.064134,0.104007,0.972321,0.767237,0.549477,0.111690,0.078765
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,...,The Four Loves,9780006280897 Lewis' work on the nature of lov...,Fiction,0.351483,0.150722,0.360707,0.251882,0.732686,0.111690,0.078765
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,...,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",Fiction,0.081412,0.184495,0.095043,0.040564,0.884389,0.475881,0.078765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,...,Mistaken Identity,9788172235222 On A Train Journey Home To North...,Nonfiction,0.148208,0.030643,0.919165,0.255172,0.853722,0.980877,0.030656
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,...,Journey to the East,9788173031014 This book tells the tale of a ma...,Fiction,0.064134,0.114382,0.051363,0.400262,0.883199,0.111690,0.227765
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,...,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...,Fiction,0.009997,0.009929,0.339219,0.947779,0.375754,0.066685,0.057625
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,...,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,Nonfiction,0.064134,0.104007,0.459269,0.759456,0.951104,0.368110,0.078765


In [25]:
books.to_csv("books_with_emotion.csv", index= False)